<a href="https://colab.research.google.com/github/AI-Junction/Nuclei-Detection/blob/master/Cell_Nuclei_Detection-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [0]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 3.8MB/s 
    100% |████████████████████████████████| 245kB 11.2MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

#filename = "/content/.kaggle/kaggle.json"
#filename = "/.kaggle/kaggle.json"
filename = "kaggle.json"
#os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!ls -lha kaggle.json

---x-wx--T 1 root root 65 Dec  4 08:39 kaggle.json


In [0]:
os.path.isdir("/root/.kaggle")

False

In [0]:
dir_kaggle = "/root/.kaggle"
if not os.path.isdir(dir_kaggle):
  !mkdir -p ~/.kaggle  
!cp kaggle.json ~/.kaggle/
os.path.isdir(dir_kaggle)

True

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!kaggle competitions files -c data-science-bowl-2018

404 - Not Found


In [0]:
if not os.path.isdir("drive/My Drive/All ML Datasets/Data-science-bowl-2018"):
  ! mkdir -p "drive/My Drive/All ML Datasets/Data-science-bowl-2018"

In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_sample_submission.csv.zip"):
  !kaggle competitions download -c data-science-bowl-2018 -f stage1_sample_submission.csv.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"
  !kaggle competitions download -c data-science-bowl-2018 -f stage1_train_labels.csv.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"
  !kaggle competitions download -c data-science-bowl-2018 -f stage1_test.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"
  !kaggle competitions download -c data-science-bowl-2018 -f stage1_train.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"
  !kaggle competitions download -c data-science-bowl-2018 -f stage2_sample_submission_final.csv.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"
  #!kaggle competitions download -c data-science-bowl-2018 -f stage2_test_final.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"
  !kaggle competitions download -c data-science-bowl-2018 -f stage1_solution.csv.zip -p "/content/drive/My Drive/All ML Datasets/Data-science-bowl-2018"

In [0]:
'''
stage1_sample_submission.csv.zip          3KB  2018-01-16 03:46:07  
stage1_train_labels.csv.zip               3MB  2018-01-16 03:46:08  
stage1_test.zip                           9MB  2018-01-16 03:46:09  
stage1_train.zip                         79MB  2018-01-16 03:46:15  
stage2_sample_submission_final.csv.zip  112KB  2018-04-12 00:03:52  
stage2_test_final.zip                   276MB  2018-04-12 00:03:54  
stage1_solution.csv.zip                 382KB  2018-04-12 00:03:57  
'''

'\nstage1_sample_submission.csv.zip          3KB  2018-01-16 03:46:07  \nstage1_train_labels.csv.zip               3MB  2018-01-16 03:46:08  \nstage1_test.zip                           9MB  2018-01-16 03:46:09  \nstage1_train.zip                         79MB  2018-01-16 03:46:15  \nstage2_sample_submission_final.csv.zip  112KB  2018-04-12 00:03:52  \nstage2_test_final.zip                   276MB  2018-04-12 00:03:54  \nstage1_solution.csv.zip                 382KB  2018-04-12 00:03:57  \n'

In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_sample_submission.csv"):
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_sample_submission.csv.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018"


In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_train_labels.csv"):
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_train_labels.csv.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018"

In [0]:
if not os.path.isdir("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test"):
  !mkdir -p "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test"
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test"

In [0]:
if not os.path.isdir("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_train"):
  !mkdir -p "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_train"
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_train.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_train"


In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_sample_submission_final.csv"):
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_sample_submission_final.csv.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018"


Archive:  drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_sample_submission_final.csv.zip
  inflating: drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_sample_submission_final.csv  


In [0]:
# not downloading stage2_test_final.zip at this stage due to large size  
'''
if not os.path.isdir("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final"):
  !mkdir -p "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final"
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final"
'''


'\nif not os.path.isdir("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final"):\n  !mkdir -p "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final"\n  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage2_test_final"\n'

In [0]:
if not os.path.isfile("drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_solution.csv"):
  !unzip "drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_solution.csv.zip" -d "drive/My Drive/All ML Datasets/Data-science-bowl-2018"


Archive:  drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_solution.csv.zip
  inflating: drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_solution.csv  


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
import theano
import os
from skimage.io import imread
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


dsb_data_dir = "drive/My Drive/All ML Datasets/Data-science-bowl-2018"

##dsb_data_dir = os.path.join('..', 'input')
stage_label = 'stage1'

In [0]:
train_labels = pd.read_csv(os.path.join(dsb_data_dir,'{}_train_labels.csv'.format(stage_label)))
train_labels['EncodedPixels'] = train_labels['EncodedPixels'].map(lambda ep: [int(x) for x in ep.split(' ')])
train_labels.sample(3)

,ImageId,EncodedPixels
7930,4217e25defac94ff465157d53f5a24b8a14045b763d860...,"[446705, 5, 447728, 7, 448752, 8, 449776, 8, 4..."
6284,33d0a9b24c25852ce35274b4b1777484ccd21f44dbe354...,"[21532, 7, 21788, 10, 22042, 12, 22296, 15, 22..."
16746,8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177...,"[8235, 4, 8753, 10, 9272, 12, 9791, 16, 10310,..."


In [0]:
os.path.join(dsb_data_dir, 'stage1_*', '*', '*', '*')

'drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_*/*/*/*'

In [0]:
all_images = glob(os.path.join(dsb_data_dir, 'stage1_*', '*', '*', '*'))

In [0]:
print(all_images[:10])

['drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test/43a71aeb641faa18742cb826772a8566c6c947d7050f9ab15459de6cc2b3b6af/images/43a71aeb641faa18742cb826772a8566c6c947d7050f9ab15459de6cc2b3b6af.png', 'drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test/a4816cc1fb76cb3c5e481186833fc0ae9cf426a1406a2607e974e65e9cddba4f/images/a4816cc1fb76cb3c5e481186833fc0ae9cf426a1406a2607e974e65e9cddba4f.png', 'drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test/1962d0c5faf3e85cda80e0578e0cb7aca50826d781620e5c1c4cc586bc69f81a/images/1962d0c5faf3e85cda80e0578e0cb7aca50826d781620e5c1c4cc586bc69f81a.png', 'drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test/4f949bd8d914bbfa06f40d6a0e2b5b75c38bf53dbcbafc48c97f105bee4f8fac/images/4f949bd8d914bbfa06f40d6a0e2b5b75c38bf53dbcbafc48c97f105bee4f8fac.png', 'drive/My Drive/All ML Datasets/Data-science-bowl-2018/stage1_test/1747f62148a919c8feb6d607faeebdf504b5e2ad42b6b1710b1189c37ebcdb2c/images/1747f62148a919c8feb6

In [0]:
#all_images = glob(os.path.join(dsb_data_dir, 'stage1_*', '*', '*', '*'))
img_df = pd.DataFrame({'path': all_images})
img_id = lambda in_path: in_path.split('/')[-3]
img_type = lambda in_path: in_path.split('/')[-2]
img_group = lambda in_path: in_path.split('/')[-4].split('_')[1]
img_stage = lambda in_path: in_path.split('/')[-4].split('_')[0]
img_df['ImageId'] = img_df['path'].map(img_id)
img_df['ImageType'] = img_df['path'].map(img_type)
img_df['TrainingSplit'] = img_df['path'].map(img_group)
img_df['Stage'] = img_df['path'].map(img_stage)
img_df.sample(2)

,path,ImageId,ImageType,TrainingSplit,Stage
6638,drive/My Drive/All ML Datasets/Data-science-bo...,351771edfc5db5665ded8aa4940257276b6526663c76e3...,masks,train,stage1
5129,drive/My Drive/All ML Datasets/Data-science-bo...,245b995878370ef4ea977568b2b67f93d4ecaa9308761b...,masks,train,stage1


In [0]:
%%time
train_df = img_df.query('TrainingSplit=="train"')
train_rows = []
group_cols = ['Stage', 'ImageId']
for n_group, n_rows in train_df.groupby(group_cols):
    c_row = {col_name: col_value for col_name, col_value in zip(group_cols, n_group)}
    c_row['masks'] = n_rows.query('ImageType == "masks"')['path'].values.tolist()
    c_row['images'] = n_rows.query('ImageType == "images"')['path'].values.tolist()
    train_rows += [c_row]

CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.28 s


In [0]:

train_img_df = pd.DataFrame(train_rows)    
IMG_CHANNELS = 3
def read_and_stack(in_img_list):
    return np.sum(np.stack([imread(c_img) for c_img in in_img_list], 0), 0)/255.0
train_img_df['images'] = train_img_df['images'].map(read_and_stack).map(lambda x: x[:,:,:IMG_CHANNELS])
train_img_df['masks'] = train_img_df['masks'].map(read_and_stack).map(lambda x: x.astype(int))
train_img_df.sample(1)

KeyboardInterrupt: ignored